In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import sys

sys.path.append("/workspace/ml-stuff/GPT")
from model import FlashAttention

import torch
import torch.nn.functional as F
import math

def test_flash_attention():
    Q, K, V = torch.randn(2, 8, 128, 64), torch.randn(2, 8, 128, 64), torch.randn(2, 8, 128, 64)
    
    # Standard attention
    standard = F.softmax(Q @ K.transpose(-1, -2) / math.sqrt(64), dim=-1) @ V
    
    # Your flash attention
    flash = FlashAttention(d=64, causal=False)(Q, K, V)
    
    assert torch.allclose(standard, flash, atol=1e-5)

test_flash_attention()

In [ ]:
import sys

sys.path.append("/workspace/ml-stuff/GPT")
from model import FlashAttention2


def test_flash_attention():
    Q, K, V = torch.randn(2, 8, 128, 64), torch.randn(2, 8, 128, 64), torch.randn(2, 8, 128, 64)
    
    # Standard attention
    standard = F.softmax(Q @ K.transpose(-1, -2) / math.sqrt(64), dim=-1) @ V
    
    # Your flash attention
    flash, _ = FlashAttention2(d=64, causal=False)(Q, K, V)
    
    assert torch.allclose(standard, flash, atol=1e-5)

test_flash_attention()

In [ ]:
import sys

sys.path.append("/workspace/ml-stuff/GPT")
from model import GPT, GPTConfig

config = GPTConfig(
    d_model=256,
    num_heads=8,
    num_layers=4,
    max_seq_len=128,
    vocab_size=1000,
    dropout=0.0,  # Important: disable dropout for comparison
    rope=True,
    rmsnorm=True,
    attn_type="standard",
)

# Create model with standard attention
model_std = GPT(config)

# Create model with SDPA, copy weights
config.attn_type = "sdpa"
model_sdpa = GPT(config)
model_sdpa.load_state_dict(model_std.state_dict())

# Test
model_std.eval()
model_sdpa.eval()

x = torch.randint(0, 1000, (2, 64))

with torch.no_grad():
    out_std = model_std(x)
    out_sdpa = model_sdpa(x)

print(f"Max diff: {(out_std - out_sdpa).abs().max().item():.2e}")
print(f"Allclose: {torch.allclose(out_std, out_sdpa, atol=1e-5)}")

Max diff: 6.56e-07
Allclose: True
